<a href="https://colab.research.google.com/github/titiodus/titiodus/blob/master/stageBquizcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
#Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import kurtosis
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

#importing the dataset
df= pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [39]:
#select a sample of the dataset to ensure linearity
simple_linear_reg_df = df[['T2', 'T6']].sample(15, random_state=2)
simple_linear_reg_df.head()

,T2,T6
1117,17.426667,-4.238889
16275,22.760000,14.690000
13272,21.230000,8.926667
3160,21.100000,7.690000
19210,21.856667,10.800000


In [40]:
#dropping columns date and lights
df.drop(['date', 'lights'], axis=1, inplace=True)
df.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [41]:
#let's normalize our dataset to a common scale using the min max scaler
scaler = MinMaxScaler()
scalerfit = scaler.fit_transform(df)
normalised_df = pd.DataFrame((scalerfit), columns=df.columns)
features_df = normalised_df.drop(columns=['Appliances'])
target_appliances = normalised_df['Appliances']

In [42]:
#Now, we split our dataset into the training and testing dataset. Recall that we had earlier segmented the features and target variables
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_df, appliances_target, test_size=0.3, random_state=42)

In [43]:
linear_model = LinearRegression()
#fit the model to the training dataset
linear_model.fit(x_train, y_train)
#obtain predictions
predicted_values = linear_model.predict(x_test)
#MAE Calculating the Mean Absolute Error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2)

0.05

In [44]:
#calcualting the Residual Sum of Squares (RSS)
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2)

45.35

In [45]:
#calculating R-Squared
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2)

0.15

In [46]:
#calculating the root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.088

In [47]:
#comparing the effects of regularization
def get_weights_df(model, feat, col_name): #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name] = weights_df[col_name].round(3)
  return weights_df



In [48]:
#Feature selection and Lasso Regression
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)
lassopredicted_values = lasso_reg.predict(x_test)

In [49]:
#estimating the RMSE of the lasso regression model with an alpha value of 0.001
lassormse = np.sqrt(mean_squared_error(y_test, lassopredicted_values))
round(lassormse, 3)

0.094

In [53]:
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-0.457,-0.411,-0.000
1,T_out,-0.322,-0.262,0.000
2,T2,-0.236,-0.201,0.000
3,T9,-0.190,-0.189,-0.000
4,RH_8,-0.158,-0.157,-0.000
5,RH_out,-0.078,-0.055,-0.050
6,RH_7,-0.045,-0.046,-0.000
7,RH_9,-0.040,-0.041,-0.000
8,T5,-0.016,-0.020,-0.000
9,T1,-0.003,-0.018,0.000


In [50]:
#Ridge Regressio
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)
ridgepredicted_values = ridge_reg.predict(x_test)

In [51]:
#estimating the RMSE of the lasso regression model with an alpha value of 0.4
ridgermse = np.sqrt(mean_squared_error(y_test, ridgepredicted_values))
round(lassormse, 3)

0.094

In [52]:
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-0.457,-0.411,-0.000
1,T_out,-0.322,-0.262,0.000
2,T2,-0.236,-0.201,0.000
3,T9,-0.190,-0.189,-0.000
4,RH_8,-0.158,-0.157,-0.000
5,RH_out,-0.078,-0.055,-0.050
6,RH_7,-0.045,-0.046,-0.000
7,RH_9,-0.040,-0.041,-0.000
8,T5,-0.016,-0.020,-0.000
9,T1,-0.003,-0.018,0.000
